In [13]:
import pandas as pd
import numpy as np
import sklearn as sk

In [26]:
#pegar so os filmes mais recorrentes
#indicar apenas com rmse abaixo de 0.5
#caso nao encontrar 3 vizinhos, dividir os filmes em 75%, senao novamente, em 50% (esses devem ser aleatorios ou os mais recentes?, talvez seja necessario ordenalos por timestamp)
#dar a opçao por content-like ou collaborative filtering
#caso nao encontrar 3 vizinhos, pegar os 75% de filmes do usuario que sao mais famosos

In [14]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [15]:
df = pd.read_csv('ml-latest/ratings.csv')

In [16]:
df

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119
...,...,...,...,...
33832157,330975,8340,2.0,1091583256
33832158,330975,8493,2.5,1091585709
33832159,330975,8622,4.0,1091581777
33832160,330975,8665,3.0,1091581765


In [5]:
popularity = (
    df.groupby("movieId")          # agrupa por filme
      .size()                      # conta avaliações por filme
      .rename("popularity")        # renomeia a série
      .reset_index()               # transforma o índice (movieId) em coluna
      .sort_values("popularity", ascending=False)  # opcional: mais populares primeiro
)

print(popularity.head())



      movieId  popularity
314       318      122296
351       356      113581
292       296      108756
2480     2571      107056
585       593      101802


In [6]:
pop = popularity.iloc[:20000,:]

In [7]:
pop['popularity'].sum()

np.int64(33479268)

In [35]:
popularity.to_csv('popularity.csv', index=False)

In [36]:
df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [17]:
df = df.iloc[:3000000, :]

In [38]:
popularity = (
    df.groupby("movieId")          # agrupa por filme
      .size()                      # conta avaliações por filme
      .rename("popularity")        # renomeia a série
      .reset_index()               # transforma o índice (movieId) em coluna
      .sort_values("popularity", ascending=False)  # opcional: mais populares primeiro
)

print(popularity.head())

      movieId  popularity
313       318       10964
350       356       10185
291       296        9759
2466     2571        9639
583       593        9119


In [ ]:
popularity.iloc[:]

,movieId,popularity
313,318,10964
350,356,10185
291,296,9759
2466,2571,9639
583,593,9119
...,...,...
36822,288295,1
36819,288209,1
36818,288207,1
36816,288163,1


In [18]:
df_pivot = df.pivot_table(index='userId', columns='movieId', values='rating', aggfunc='mean')

In [19]:
df_pivot

movieId,1,2,3,4,5,6,7,8,9,10,...,288745,288749,288753,288755,288757,288765,288851,288939,288941,288957
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.0,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,4.0,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_pivot = pd.read_csv("project/df_pivot.csv", index_col=0, dtype=np.float32)

In [27]:
import pandas as pd

def exportar_popularidade(df_pivot, caminho_csv="popularidade_filmes.csv"):
    """
    Cria um CSV com movieId e quantidade de avaliações (do mais popular ao menos).
    """
    # Conta quantos usuários avaliaram cada filme (não NaN)
    popularidade = df_pivot.notna().sum().sort_values(ascending=False)

    # Transforma em DataFrame
    df_popularidade = pd.DataFrame({
        "movieId": popularidade.index,
        "quantidade_avaliacoes": popularidade.values
    })

    # Salva em CSV
    df_popularidade.to_csv(caminho_csv, index=False, encoding="utf-8")
    print(f"📂 Arquivo salvo em: {caminho_csv}")

    return df_popularidade


# Exemplo de uso:
df_pop = exportar_popularidade(df_pivot, "popularidade_filmes.csv")
df_pop.head()


📂 Arquivo salvo em: popularidade_filmes.csv


,movieId,quantidade_avaliacoes
0,318,10964
1,356,10185
2,296,9759
3,2571,9639
4,593,9119


In [ ]:
from scipy.sparse import csr_matrix
knn = NearestNeighbors(metric='cosine', algorithm='brute')

for idx, row in df_pivot.iterrows():
    # Selecionar apenas as colunas com avaliações (sem NaN)
    row_valid = row.dropna()
    
    # Se houver menos de 2 filmes avaliados, pula
    if len(row_valid) < 3:
        continue

    # Filmes avaliados por esse usuário
    filmes = row_valid.index.tolist()

    # Filtrar DataFrame com apenas esses filmes
    validos = df_pivot[filmes].dropna(how='any')  # Apenas usuários que avaliaram TODOS esses filmes
    
    # Remover o usuário atual
    validos = validos.drop(index=idx, errors='ignore')
    
    # Se não houver outros usuários com essas mesmas avaliações, pula
    if validos.shape[0] == 0:
        continue

    # Criar matriz esparsa
    matrix_sparse = csr_matrix(validos.values)
    user_vector = csr_matrix(row_valid.values.reshape(1, -1))
    if matrix_sparse.shape[0] >= 3:
        knn.fit(matrix_sparse)
        distances, indices = knn.kneighbors(user_vector, n_neighbors=3, return_distance=True)
        distance = np.array(distances[0])
        print(f' {distance} - vizinhos: {indices.flatten()}')
       
       
    else:
        print(f'Usuário {idx} ignorado - não há amostras suficientes para KNN')
    # Treinar modelo
        knn.fit(matrix_sparse)


Usuário 2 ignorado - não há amostras suficientes para KNN
 [4.47213595 5.67890835 6.        ] - vizinhos: [5 3 6]
Usuário 6 ignorado - não há amostras suficientes para KNN
Usuário 8 ignorado - não há amostras suficientes para KNN
 [4.         4.0620192  4.15331193] - vizinhos: [3 8 5]
 [5.40832691 5.70087713 5.87367006] - vizinhos: [14  5  8]
Usuário 16 ignorado - não há amostras suficientes para KNN
 [0.70710678 1.87082869 1.87082869] - vizinhos: [5 3 4]
 [0.8660254 0.8660254 1.       ] - vizinhos: [14 34 39]
 [0.8660254  1.         1.22474487] - vizinhos: [ 60 101  83]
 [6.08276253 7.05336799 9.74679434] - vizinhos: [2 1 0]
 [2.82842712 3.96862697 4.30116263] - vizinhos: [1 0 4]
Usuário 29 ignorado - não há amostras suficientes para KNN
 [0.8660254  1.         1.11803399] - vizinhos: [ 15 287 241]
 [1.11803399 1.11803399 1.22474487] - vizinhos: [ 7  9 14]
 [2.64575131 2.6925824  2.82842712] - vizinhos: [16  9  3]
 [0.  0.5 0.5] - vizinhos: [430 423 159]
 [1.41421356 1.41421356 1.4142

KeyboardInterrupt: 

In [40]:
ratings = {296: 4.0, 318: 5.0, 356: 4.0, 296: 3.0, 2571: 4.0, 593: 5.0, 260: 4.0, 104218: 2.0, 160573: 4.5, 4256: 3.5, 8363: 2.0, 32: 4.5}

In [55]:
df_pivot

,userId,movieId,rating,timestamp
index,,,,
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119
...,...,...,...,...
2999995,29506,4790,4.0,1005621448
2999996,29506,4797,4.0,1084742722
2999997,29506,4799,4.0,1005621448


In [26]:
df_pivot = df_pivot.astype(np.float32)
df_pivot.columns = df_pivot.columns.astype(str)
df_pivot.to_parquet("project/df_pivot.parquet", engine="fastparquet", index=True)


In [6]:
df_tent = pd.read_parquet("project/df_pivot.parquet", engine="fastparquet")

In [8]:
col = df_tent.columns

In [12]:
col
    

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       ...
       '288745', '288749', '288753', '288755', '288757', '288765', '288851',
       '288939', '288941', '288957'],
      dtype='object', length=36850)

In [20]:
popularidade = df_pivot.notna().sum().sort_values(ascending=False)

In [21]:
popularidade = pd.DataFrame({
    "movieId": popularidade.index,
    "popularidade": popularidade.values
})

In [22]:
popularidade.to_csv('project/popularidade_aplicacao.csv', index=True)

In [50]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors


def recomendar_usuario(df_pivot, meus_ratings, limiar_sim=0.7, random_state=42, caminho_popularidade="popularidade_aplicacao.csv"):
    """
    df_pivot: DataFrame pivotado userId x movieId com ratings (1-5, NaN se não avaliou)
    meus_ratings: dict {movieId: rating}
    limiar_sim: float, limiar mínimo de similaridade cosine
    caminho_popularidade: str, caminho para o CSV com colunas [movieId, popularidade]
    """
    np.random.seed(random_state)
    
    # Cria a linha do usuário
    minha_linha = pd.Series(meus_ratings, name='meu_usuario')
    
    # Lista de IDs dos filmes que forneceu
    meus_filmes = list(meus_ratings.keys())
    
    # 🔹 Lê o CSV de popularidade
    df_pop = pd.read_csv(caminho_popularidade)
    popularidade = df_pop.set_index("movieId")["popularidade"].to_dict()
    
    # Estratégias de filtragem: (percentual de filmes, modo)
    estrategias = [
        (1.0, 'todos'),
        (0.75, 'aleatorio'),
        (0.75, 'populares'),
        (0.5, 'aleatorio'),
        (0.5, 'populares'),
        (0.25, 'aleatorio'),
        (0.25, 'populares')
    ]
    
    melhor_usuario = None
    melhor_sim = -1
    vizinhos_extra = []

    for perc, modo in estrategias:
        print(f"\n🔎 Testando estratégia: {modo.upper()} ({int(perc*100)}% dos filmes)")

        # Escolher filmes para a rodada
        if perc < 1.0:
            n_filmes = max(1, int(len(meus_filmes) * perc))
            if modo == 'aleatorio':
                filmes_selecionados = list(np.random.choice(meus_filmes, size=n_filmes, replace=False))
            elif modo == 'populares':
                filmes_ordenados = [f for f in meus_filmes if f in popularidade]
                filmes_ordenados.sort(key=lambda x: popularidade[x], reverse=True)
                filmes_selecionados = filmes_ordenados[:n_filmes]
        else:
            filmes_selecionados = meus_filmes
        
        print(f"🎬 Filmes usados nesta rodada: {filmes_selecionados}")

        # Filtra usuários que avaliaram todos os filmes selecionados
        df_filtrado = df_pivot.dropna(subset=filmes_selecionados, how='any')
        print(f"📊 Usuários restantes após filtro: {df_filtrado.shape[0]}")
        
        if df_filtrado.shape[0] == 0:
            print("⚠️ Nenhum usuário encontrado para essa seleção de filmes.")
            continue
        
        # Construir matriz para cálculo de similaridade
        X = df_filtrado[filmes_selecionados].values
        minha_vetor = minha_linha[filmes_selecionados].values.reshape(1, -1)
        
        # KNN cosine (pegar até 4 vizinhos: 1 principal + 3 extras)
        n_vizinhos = min(4, df_filtrado.shape[0])  
        knn = NearestNeighbors(n_neighbors=n_vizinhos, metric='cosine')
        knn.fit(X)
        dist, ind = knn.kneighbors(minha_vetor, n_neighbors=n_vizinhos)
        
        dist = dist[0]
        ind = ind[0]
        
        # Calcular similaridades
        sims = 1 - dist
        usuarios = df_filtrado.index[ind].tolist()
        
        print("👥 Vizinhos encontrados:")
        for u, s in zip(usuarios, sims):
            print(f"   - {u} (similaridade={s:.4f})")
        
        # Melhor usuário (primeiro da lista)
        sim = sims[0]
        usuario_encontrado = usuarios[0]
        
        if sim > limiar_sim:
            print("✅ Similaridade acima do limiar, parando busca.")
            melhor_usuario = usuario_encontrado
            melhor_sim = sim
            vizinhos_extra = list(zip(usuarios[1:], sims[1:]))  # próximos 3
            break
        else:
            print("❌ Similaridade baixa, tentando próxima estratégia...")

    return {
        'usuario_mais_similar': melhor_usuario,
        'similaridade': melhor_sim,
        'filmes_usados': meus_filmes,
        'vizinhos_extras': vizinhos_extra  # lista de (usuario, similaridade)
    }


# Exemplo de uso:
result = recomendar_usuario(df_pivot, ratings, limiar_sim=0.7)
print(result)


FileNotFoundError: [Errno 2] No such file or directory: 'popularidade_aplicacao.csv'

In [42]:
def recomendar_filmes(df_pivot, resultado_recomendar, meus_ratings, top_n=3, min_rating=4):
    """
    df_pivot: DataFrame pivotado userId x movieId com ratings
    resultado_recomendar: dict retornado pela função recomendar_usuario
    meus_ratings: dict {movieId: rating} -> filmes já assistidos pelo usuário
    top_n: quantidade de filmes a recomendar
    min_rating: nota mínima para recomendar
    
    Retorna: lista de movieIds recomendados
    """
    candidatos = [resultado_recomendar["usuario_mais_similar"]] + resultado_recomendar.get("vizinhos_extras", [])
    filmes_assistidos = set(meus_ratings.keys())
    
    for user_id in candidatos:
        if user_id is None:
            continue
        
        print(f"\n🎯 Tentando recomendar filmes do usuário {user_id}...")
        linha_usuario = df_pivot.loc[user_id]
        
        # Ordena os filmes avaliados pelo usuário (maior rating primeiro)
        filmes_ordenados = (
            linha_usuario.dropna()
            .sort_values(ascending=False)
        )
        
        # Filtra apenas filmes não assistidos e com rating >= min_rating
        filmes_validos = [mid for mid, nota in filmes_ordenados.items() 
                          if mid not in filmes_assistidos and nota >= min_rating]
        
        if len(filmes_validos) >= top_n:
            recomendados = filmes_validos[:top_n]
            print(f"✅ Filmes recomendados: {recomendados}")
            return recomendados
        else:
            print(f"⚠️ Usuário {user_id} não tem filmes suficientes (>= {min_rating}) para recomendar.")
    
    print("❌ Nenhum usuário tinha recomendações válidas.")
    return []


In [43]:
# Primeiro encontra o usuário mais próximo
resultado = recomendar_usuario(df_pivot, ratings, limiar_sim=0.7)

# Depois busca os filmes recomendados
recomendados = recomendar_filmes(df_pivot, resultado, ratings, top_n=3, min_rating=4)

print("🎬 Recomendação final:", recomendados)



🔎 Testando estratégia: TODOS (100% dos filmes)
🎬 Filmes usados nesta rodada: [296, 318, 356, 2571, 593, 260, 104218, 160573, 4256, 8363, 32]
📊 Usuários restantes após filtro: 0
⚠️ Nenhum usuário encontrado para essa seleção de filmes.

🔎 Testando estratégia: ALEATORIO (75% dos filmes)
🎬 Filmes usados nesta rodada: [np.int64(260), np.int64(296), np.int64(8363), np.int64(32), np.int64(356), np.int64(318), np.int64(4256), np.int64(593)]
📊 Usuários restantes após filtro: 0
⚠️ Nenhum usuário encontrado para essa seleção de filmes.

🔎 Testando estratégia: POPULARES (75% dos filmes)
🎬 Filmes usados nesta rodada: [318, 356, 296, 2571, 593, 260, 32, 104218]
📊 Usuários restantes após filtro: 26
👥 Vizinhos encontrados:
   - 10730 (similaridade=0.9952)
   - 527 (similaridade=0.9870)
   - 11299 (similaridade=0.9869)
   - 15227 (similaridade=0.9863)
✅ Similaridade acima do limiar, parando busca.

🎯 Tentando recomendar filmes do usuário 10730...
✅ Filmes recomendados: [5072, 4993, 205156]
🎬 Recomend

In [22]:
df_pivot.columns

Index([     1,      2,      3,      4,      5,      6,      7,      8,      9,
           10,
       ...
       288745, 288749, 288753, 288755, 288757, 288765, 288851, 288939, 288941,
       288957],
      dtype='int64', name='movieId', length=36850)

In [23]:
import pandas as pd

# Lê os CSVs
movies = pd.read_csv("ml-latest/movies.csv")  # contém movieId e title
popularity = pd.read_csv("project/popularidade_aplicacao.csv")  # contém movieId

# Faz o merge (inner join) usando o movieId
filmes_usados = pd.merge(popularity, movies, on="movieId", how="inner")

# Salva o resultado
filmes_usados.to_csv("project/filmes_usados.csv", index=False)

print("CSV 'filmes_usados.csv' criado com sucesso!")


CSV 'filmes_usados.csv' criado com sucesso!


In [ ]:
from scipy.sparse import csr_matrix
import numpy as np
knn = NearestNeighbors(metric='euclidean', algorithm='brute')

# Define a threshold para RMSE (por exemplo, 1.0)
rmse_threshold = 1.0

for idx, row in df_pivot.iterrows():
    # Selecionar apenas as colunas com avaliações (sem NaN)
    row_valid = row.dropna()
    
    # Se houver menos de 3 filmes avaliados, pula
    if len(row_valid) < 3:
        continue

    # Filmes avaliados por esse usuário
    filmes = row_valid.index.tolist()

    # Filtrar DataFrame com apenas esses filmes
    validos = df_pivot[filmes].dropna(how='any')  # Apenas usuários que avaliaram TODOS esses filmes
    
    # Remover o usuário atual
    validos = validos.drop(index=idx, errors='ignore')
    
    # Se não houver outros usuários com essas mesmas avaliações, pula
    if validos.shape[0] == 0:
        continue

    # Criar matriz esparsa
    matrix_sparse = csr_matrix(validos.values)
    user_vector = csr_matrix(row_valid.values.reshape(1, -1))

    if matrix_sparse.shape[0] >= 3:
        knn.fit(matrix_sparse)
        distances, indices = knn.kneighbors(user_vector, n_neighbors=3, return_distance=True)
        
        # Calcular a distância Euclidiana
        distance = np.array(distances[0])
        
        # Normalizar a distância Euclidiana por k (número de filmes em comum)
        k = len(filmes)  # Número de filmes avaliados
        rmse_per_item = distance / np.sqrt(k)

        # Imprimir as distâncias e RMSE
        

        # Filtrar vizinhos com base no threshold de RMSE
        valid_neighbors = [i for i, rmse in zip(indices.flatten(), rmse_per_item) if rmse <= rmse_threshold]
        
        if valid_neighbors:
            print(f'Usuário {idx} tem vizinhos válidos (RMSE <= {rmse_threshold}): {valid_neighbors}')
        else:
            print(f'Usuário {idx} não tem vizinhos válidos com RMSE <= {rmse_threshold}')
        
    else:
        print(f'Usuário {idx} ignorado - não há amostras suficientes para KNN')



Usuário 2 ignorado - não há amostras suficientes para KNN
Usuário 3 tem vizinhos válidos (RMSE <= 1.0): [np.int64(5)]
Usuário 6 ignorado - não há amostras suficientes para KNN
Usuário 8 ignorado - não há amostras suficientes para KNN
Usuário 11 tem vizinhos válidos (RMSE <= 1.0): [np.int64(3), np.int64(8), np.int64(5)]
Usuário 15 tem vizinhos válidos (RMSE <= 1.0): [np.int64(14), np.int64(5), np.int64(8)]
Usuário 16 ignorado - não há amostras suficientes para KNN
Usuário 19 tem vizinhos válidos (RMSE <= 1.0): [np.int64(5), np.int64(3), np.int64(4)]
Usuário 20 tem vizinhos válidos (RMSE <= 1.0): [np.int64(14), np.int64(34), np.int64(39)]
Usuário 23 tem vizinhos válidos (RMSE <= 1.0): [np.int64(60), np.int64(101), np.int64(83)]
Usuário 25 não tem vizinhos válidos com RMSE <= 1.0
Usuário 26 tem vizinhos válidos (RMSE <= 1.0): [np.int64(1), np.int64(0), np.int64(4)]
Usuário 29 ignorado - não há amostras suficientes para KNN
Usuário 32 tem vizinhos válidos (RMSE <= 1.0): [np.int64(15), np.i